In [1]:

import Impute

In [31]:
using Plots
using DataFrames
import CSV
using MLJ
using LossFunctions

In [3]:
features = DataFrame(CSV.File("features.csv",missingstring="missing"));
train = DataFrame(CSV.File("train.csv"));
test = DataFrame(CSV.File("test.csv"));
stores = DataFrame(CSV.File("stores.csv"));
features.StoreType = stores[features.Store,:Type];
features.Size = stores[features.Store,:Size];
features.Id1 = string.(features.Store,"_",features.Date);
select!(features, Not([:Store,:Date]));

In [4]:
# train.Id = string.(train.Store,"_",train.Dept,"_",train.Date);
train.Id1 = string.(train.Store,"_",train.Date);
# select!(train, Not([:Store,:Date,:IsHoliday]));
# select!(train, :Id,:Weekly_Sales,:IsHoliday);

# test.Id = string.(test.Store,"_",test.Dept,"_",test.Date);
test.Id1 = string.(test.Store,"_",test.Date);
# select!(test, Not([:Store,:Date,:IsHoliday]));
# select!(test, :Id,:IsHoliday);

In [5]:
train = innerjoin(features,train,on=:Id1,makeunique=true)
train.Id = string.(train.Store,"_",train.Dept,"_",train.Date);
# select!(train, Not([:Store,:Date,:IsHoliday,:Id1]));
# target = select(train, :Weekly_Sales);
select!(train, Not([:Id1,:IsHoliday_1]));
train;

In [6]:
test = innerjoin(features,test,on=:Id1,makeunique=true)
test.Id = string.(test.Store,"_",test.Dept,"_",test.Date);
# select!(test, Not([:Store,:Date,:IsHoliday,:Id1]));
select!(test, Not([:Id1,:IsHoliday_1]));
test;

In [7]:
train = coerce(train, :StoreType => Multiclass);

In [8]:
schema(train)

┌──────────────┬──────────────────────────────────┬─────────────────────────────
│ _.names      │ _.types                          │ _.scitypes                 ⋯
├──────────────┼──────────────────────────────────┼─────────────────────────────
│ Temperature  │ Float64                          │ Continuous                 ⋯
│ Fuel_Price   │ Float64                          │ Continuous                 ⋯
│ MarkDown1    │ Union{Missing, Float64}          │ Union{Missing, Continuous} ⋯
│ MarkDown2    │ Union{Missing, Float64}          │ Union{Missing, Continuous} ⋯
│ MarkDown3    │ Union{Missing, Float64}          │ Union{Missing, Continuous} ⋯
│ MarkDown4    │ Union{Missing, Float64}          │ Union{Missing, Continuous} ⋯
│ MarkDown5    │ Union{Missing, Float64}          │ Union{Missing, Continuous} ⋯
│ CPI          │ Union{Missing, Float64}          │ Union{Missing, Continuous} ⋯
│ Unemployment │ Union{Missing, Float64}          │ Union{Missing, Continuous} ⋯
│ IsHoliday    │ Bool       

In [9]:
train = Impute.filter(train; dims=:rows) # drop missing rows
# train = Impute.substitute(train) # substitute (somehow!?) missing data

,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,59.11,3.297,10382.9,6115.67,215.07,2406.62,6551.42
2,62.25,3.308,6074.12,254.39,51.98,427.39,5988.57
3,60.14,3.236,410.31,98.0,55805.5,8.0,554.92
4,48.91,3.172,5629.51,68.0,1398.11,2084.64,20475.3
5,43.93,3.158,4640.65,19.0,105.02,3639.42,14461.8
6,51.63,3.159,5011.32,67.0,347.37,225.79,4011.37
7,47.96,3.112,2725.36,40.48,634.7,24.9,2739.43
8,44.55,3.129,5762.1,46011.4,260.36,983.65,4735.78
9,49.01,3.157,6277.39,21813.2,143.1,1450.13,8483.0


In [10]:
# y, X = unpack(train, ==(:Weekly_Sales), colname -> true)
y, X = unpack(train, 
    ==(:Weekly_Sales), 
    colname -> colname != :Date &&  colname !=:Id
#     !=([:Date, :Id])
#     !=(:Date)
    );

In [11]:
schema(X)

┌──────────────┬──────────────────────────────────┬───────────────┐
│ _.names      │ _.types                          │ _.scitypes    │
├──────────────┼──────────────────────────────────┼───────────────┤
│ Temperature  │ Float64                          │ Continuous    │
│ Fuel_Price   │ Float64                          │ Continuous    │
│ MarkDown1    │ Float64                          │ Continuous    │
│ MarkDown2    │ Float64                          │ Continuous    │
│ MarkDown3    │ Float64                          │ Continuous    │
│ MarkDown4    │ Float64                          │ Continuous    │
│ MarkDown5    │ Float64                          │ Continuous    │
│ CPI          │ Float64                          │ Continuous    │
│ Unemployment │ Float64                          │ Continuous    │
│ IsHoliday    │ Bool                             │ Count         │
│ StoreType    │ CategoricalValue{String, UInt32} │ Multiclass{3} │
│ Size         │ Int64                          

In [12]:
models(matching(X, y))

4-element Vector{NamedTuple{(:name, :package_name, :is_supervised, :docstring, :hyperparameter_ranges, :hyperparameter_types, :hyperparameters, :implemented_methods, :is_pure_julia, :is_wrapper, :iteration_parameter, :load_path, :package_license, :package_url, :package_uuid, :prediction_type, :supports_class_weights, :supports_online, :supports_training_losses, :supports_weights, :input_scitype, :target_scitype, :output_scitype), T} where T<:Tuple}:
 (name = ConstantRegressor, package_name = MLJModels, ... )
 (name = DecisionTreeRegressor, package_name = BetaML, ... )
 (name = DeterministicConstantRegressor, package_name = MLJModels, ... )
 (name = RandomForestRegressor, package_name = BetaML, ... )

In [18]:
@load DecisionTreeRegressor pkg="DecisionTree"
regressor = MLJDecisionTreeInterface.DecisionTreeRegressor()

import MLJDecisionTreeInterface ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/alex/.julia/packages/MLJModels/E8BbE/src/loading.jl:168


DecisionTreeRegressor(
    max_depth = -1,
    min_samples_leaf = 5,
    min_samples_split = 2,
    min_purity_increase = 0.0,
    n_subfeatures = 0,
    post_prune = false,
    merge_purity_threshold = 1.0) @235

In [21]:
pipe = @pipeline(Standardizer,
                 OneHotEncoder,
                 regressor )

Pipeline262(
    standardizer = Standardizer(
            features = Symbol[],
            ignore = false,
            ordered_factor = false,
            count = false),
    one_hot_encoder = OneHotEncoder(
            features = Symbol[],
            drop_last = false,
            ordered_factor = true,
            ignore = false),
    decision_tree_regressor = DecisionTreeRegressor(
            max_depth = -1,
            min_samples_leaf = 5,
            min_samples_split = 2,
            min_purity_increase = 0.0,
            n_subfeatures = 0,
            post_prune = false,
            merge_purity_threshold = 1.0)) @477

In [22]:
regress = machine(pipe, X, y)

Machine{Pipeline262,…} @150 trained 0 times; caches data
  args: 
    1:	Source @185 ⏎ `Table{Union{AbstractVector{Continuous}, AbstractVector{Count}, AbstractVector{Multiclass{3}}}}`
    2:	Source @545 ⏎ `AbstractVector{Continuous}`


In [23]:
train, test = partition(eachindex(y), 0.7, shuffle=true, rng=0)

([12585, 28911, 6448, 87930, 72097, 58627, 96392, 55000, 7974, 21168  …  92403, 59988, 50528, 18412, 91173, 27475, 38297, 89570, 67786, 53080], [71979, 7797, 32381, 79326, 50233, 38904, 48203, 51548, 3284, 55709  …  2444, 68687, 71423, 21785, 24288, 44779, 86417, 89142, 25498, 73281])

In [24]:
fit!(regress, rows=train)

┌ Info: Training Machine{Pipeline262,…} @150.
└ @ MLJBase /Users/alex/.julia/packages/MLJBase/hLtde/src/machines.jl:342
┌ Info: Training Machine{Standardizer,…} @759.
└ @ MLJBase /Users/alex/.julia/packages/MLJBase/hLtde/src/machines.jl:342
┌ Info: Training Machine{OneHotEncoder,…} @643.
└ @ MLJBase /Users/alex/.julia/packages/MLJBase/hLtde/src/machines.jl:342
┌ Info: Spawning 3 sub-features to one-hot encode feature :StoreType.
└ @ MLJModels /Users/alex/.julia/packages/MLJModels/E8BbE/src/builtins/Transformers.jl:1142
┌ Info: Training Machine{DecisionTreeRegressor,…} @992.
└ @ MLJBase /Users/alex/.julia/packages/MLJBase/hLtde/src/machines.jl:342


Machine{Pipeline262,…} @150 trained 1 time; caches data
  args: 
    1:	Source @185 ⏎ `Table{Union{AbstractVector{Continuous}, AbstractVector{Count}, AbstractVector{Multiclass{3}}}}`
    2:	Source @545 ⏎ `AbstractVector{Continuous}`


In [26]:
fitted_params(regress) |> println

(standardizer = Dict(:CPI => (174.78910061423335, 39.656137408654544), :Fuel_Price => (3.6192831510619823, 0.2799605459670509), :MarkDown1 => (8822.068469509413, 9247.185701195871), :MarkDown3 => (1787.7802499300844, 10917.794131475586), :Temperature => (57.30800468066942, 18.276825620533234), :MarkDown5 => (5304.424938106243, 6538.396397750151), :MarkDown4 => (4020.609798790091, 7170.4930882228555), :MarkDown2 => (3682.778021166046, 10017.908111361498), :Unemployment => (7.413088123169314, 1.7080678924323616)), one_hot_encoder = (fitresult = OneHotEncoderResult @311,), decision_tree_regressor = (tree = Decision Tree
Leaves: 10915
Depth:  29,), machines = Machine[Machine{Standardizer,…} @759, Machine{OneHotEncoder,…} @643, Machine{DecisionTreeRegressor,…} @992], fitted_params_given_machine = OrderedCollections.LittleDict{Any, Any, Vector{Any}, Vector{Any}}(Machine{Standardizer,…} @759 => Dict(:CPI => (174.78910061423335, 39.656137408654544), :Fuel_Price => (3.6192831510619823, 0.279960

In [42]:
ypred = predict(regress, rows=test)


29117-element Vector{Float64}:
 19319.748571428576
    30.318
  4699.126
 60253.881250000006
 39434.036666666674
   180.21444444444444
 20168.379999999997
  1745.6516666666666
 23435.325999999997
   240.78142857142856
  1896.6155555555556
 29315.308
  4451.016
     ⋮
  3007.53
  3289.2033333333334
  5873.653333333333
   417.0
  2820.215714285714
  2658.9100000000003
 73145.27625
 52971.58499999999
 21890.21571428571
  8264.966
  7331.273999999999
  7418.4087500000005

In [49]:
y[test]

29117-element Vector{Float64}:
 19090.39
    83.13
  3187.41
 58371.33
 37473.21
   144.78
 18933.78
  1933.25
 21620.99
   229.21
  1224.0
 25756.91
   488.0
     ⋮
  2399.74
  3015.06
  4983.9
  1215.0
  2745.98
  1091.32
 69753.8
 49737.78
 24225.53
  6267.34
  8215.38
  6988.66

In [47]:
LossFunctions.value(L2DistLoss(), y[test], ypred)

29117-element Vector{Float64}:
  52605.35428775738
   2789.1073439999996
      2.2852852646560012e6
      3.5439992088765777e6
      3.844841216711144e6
   1255.5998530864192
      1.5242371599999964e6
  35193.13466944445
      3.2918151208959846e6
    133.8979591836731
 452411.6855753087
      1.2662196326404007e7
      1.5705495816255998e7
      ⋮
 369408.68410000054
  75154.56721111116
 791660.9941777773
 636804.0
   5510.9412755101675
      2.457338408100001e6
      1.1502111154314011e7
      1.0457494778024955e7
      5.453692813061237e6
      3.990509635876001e6
 781643.4192359996
 184683.98812656305